# Getting started

A simple notebook to illustrate how to use the _actymath_ package to build quick actuarial calculations.


In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)


## Fetching mortality data

The first element is extracting the relevant mortality probabilities `q(x)` for the person or person.

For this example:

- A 30 year old
- We will use an old Actuarial A1967-70 table
- Select mortality


In [3]:
from actymath.tables import A1967_70

table = A1967_70()
table.__doc__


'UK A1967-70 mortality table provided by the CMI usese Qx in diagonal select pattern.\n    https://www.actuaries.org.uk/learn-and-develop/continuous-mortality-investigation/cmi-mortality-and-morbidity-tables/mortality-rates-older-mortality-tables\n    '

The mortality rates can be fetched into a list for a particular age using the `.qx(age=x, select=False)` method.

This returns a list with the first rate being the first element in the list.


In [4]:
qx = table.qx(age=30)
print(qx)


[0.00043767, 0.00057371, 0.00069882, 0.00073813, 0.00079004, 0.00085577, 0.00093663, 0.00103409, 0.00114973, 0.00128528, 0.00144267, 0.00162396, 0.00183145, 0.00206763, 0.00233523, 0.00263723, 0.00297691, 0.00335783, 0.00378388, 0.00425932, 0.0047888, 0.0053774, 0.00603064, 0.00675456, 0.00755572, 0.00844128, 0.00941902, 0.01049742, 0.01168566, 0.01299373, 0.01443246, 0.01601356, 0.01774972, 0.01965464, 0.0217431, 0.02403101, 0.0265355, 0.02927491, 0.0322689, 0.03553846, 0.03910594, 0.04299507, 0.04723096, 0.05184008, 0.05685022, 0.06229041, 0.06819083, 0.07458263, 0.08149779, 0.08896883, 0.09702855, 0.10570968, 0.11504443, 0.12506403, 0.1357982, 0.14727448, 0.15951762, 0.17254883, 0.18638498, 0.20103786, 0.21651335, 0.23281066, 0.2499216, 0.2678299, 0.28651074, 0.30593028, 0.3260455, 0.34680421, 0.36814521, 0.38999883, 0.41228762, 0.43492729, 0.45782791, 0.48089531, 0.50403251, 0.52714144, 0.55012452, 0.57288631, 0.5953351, 0.6173843, 0.63895372, 0.6599706, 0.68037042, 0.70009748, 0.7

## The Calc object

The calculation is done using an expanded version of a pandas DataFrame.

To start a calculation you need to create a new `Calc()` instance.


In [5]:
from actymath import Calc

calc = Calc()
calc.shape


(0, 0)

The `calc` object is initially empty and has a number of additional methods that enable us to quickly build up an actuarial calculation.

First we should always add in the lives and their mortality. We can use the special `Calc.add_life()` method for this.


In [6]:
calc.add_life(age=30, qx=qx)


'x1'

In [7]:
calc


,q(x1),x1
0,0.000438,30
1,0.000574,31
2,0.000699,32
3,0.000738,33
4,0.000790,34
...,...,...
87,0.771483,117
88,0.787329,118
89,0.802354,119
90,0.816564,120


We could add multiple lives (x2, x3, ...) to build up joint life products. In this case we just have this one life.

Next we can add in the 3.25% calculation interest rate with the `Calc.add_i()` method


In [8]:
calc.add_i(rate=0.0325)
calc.head()


,q(x1),x1,i
0,0.000438,30,0.0325
1,0.000574,31,0.0325
2,0.000699,32,0.0325
3,0.000738,33,0.0325
4,0.000790,34,0.0325


Policies usually also have a number of different terms (durations) that are used to define the policy term, premium term, etc. We can add multiple different terms (n1, n2, ...) to the calc that we can use in the calculations.

This is done easily using the `Calc.add_term()` method.

This policy will have a 15 year policy term but only a 10 year premium paying term.


In [9]:
calc.add_term(n=15)


'n1'

In [10]:
calc.add_term(n=10)


'n2'

The 'n1' and 'n1' colums added for these two term durations provide a 'remaining periods' counter that is used by the actuarial calculations.


In [11]:
calc.head()


,q(x1),x1,i,n1,n2
0,0.000438,30,0.0325,15.0,10.0
1,0.000574,31,0.0325,14.0,9.0
2,0.000699,32,0.0325,13.0,8.0
3,0.000738,33,0.0325,12.0,7.0
4,0.000790,34,0.0325,11.0,6.0


We have the basic calculation time grid set up. Because the `calc` object is just a pandas.DataFrame under the hood, you can use all of the usual pandas methods to slice, view and manipulate it if you like.

Also as it is based upon pandas, it uses C for the calculations and uses vectorised calcs so the calculation should be very fast.


In [12]:
calc.iloc[30]


q(x1)     0.014432
x1       60.000000
i         0.032500
n1             NaN
n2             NaN
Name: 30, dtype: float64

## Adding actuarial benefits

We will first add a term Endowment assurance benefit on the life (x1) for the 15 year term (n1).

We add new formulae or benefits using the `Calc.populate(column)` method.


In [13]:
calc.populate("EA(x1)[n1]")
calc.head()


,q(x1),x1,i,n1,n2,v^t,l(x1),d(x1),C(x1),M(x1),D(x1),EA(x1)[n1]
0,0.000438,30,0.0325,15.0,10.0,1.000000,34481.408000,15.091478,14.616443,8719.096980,34481.408000,0.620982
1,0.000574,31,0.0325,14.0,9.0,0.968523,34466.316522,19.773670,18.548431,8704.480537,33381.420360,0.641007
2,0.000699,32,0.0325,13.0,8.0,0.938037,34446.542852,24.071933,21.869597,8685.932106,32312.125042,0.661645
3,0.000738,33,0.0325,12.0,7.0,0.908510,34422.470919,25.408258,22.357058,8664.062509,31273.166763,0.682927
4,0.000790,34,0.0325,11.0,6.0,0.879913,34397.062660,27.175055,23.159018,8641.705451,30266.424310,0.704905


The populate function knows what that formula column depends upon and calculates those missing dependencies automatically. In this case it is the D, C and M commutation functions that are required.


Now let's add the limited premium term formula for the 10 years (n2).

It will also add in any further missing dependencies it needs - in this case the N commutation function.


In [14]:
calc.populate("a_due(x1)[n2]")
calc.head(20)


,q(x1),x1,i,n1,n2,v^t,l(x1),d(x1),C(x1),M(x1),D(x1),EA(x1)[n1],N(x1),a_due(x1)[n2]
0,0.000438,30,0.0325,15.0,10.0,1.000000,34481.408000,15.091478,14.616443,8719.096980,34481.408000,0.620982,818448.803936,8.670826
1,0.000574,31,0.0325,14.0,9.0,0.968523,34466.316522,19.773670,18.548431,8704.480537,33381.420360,0.641007,783967.395936,7.923596
2,0.000699,32,0.0325,13.0,8.0,0.938037,34446.542852,24.071933,21.869597,8685.932106,32312.125042,0.661645,750585.975575,7.152716
3,0.000738,33,0.0325,12.0,7.0,0.908510,34422.470919,25.408258,22.357058,8664.062509,31273.166763,0.682927,718273.850533,6.357122
4,0.000790,34,0.0325,11.0,6.0,0.879913,34397.062660,27.175055,23.159018,8641.705451,30266.424310,0.704905,687000.683770,5.535314
5,0.000856,35,0.0325,10.0,5.0,0.852216,34369.887605,29.412719,24.276988,8618.546433,29290.569128,0.727599,656734.259460,4.686415
6,0.000937,36,0.0325,9.0,4.0,0.825391,34340.474886,32.164319,25.712478,8594.269445,28344.312966,0.751033,627443.690333,3.809483
7,0.001034,37,0.0325,8.0,3.0,0.799410,34308.310567,35.477881,27.468642,8568.556966,27426.406617,0.775231,599099.377367,2.903511
8,0.001150,38,0.0325,7.0,2.0,0.774247,34272.832686,39.404504,29.548492,8541.088324,26535.637040,0.800219,571672.970749,1.967409
9,0.001285,39,0.0325,6.0,1.0,0.749876,34233.428182,43.999541,31.955641,8511.539832,25670.826366,0.826026,545137.333709,1.000000


Because the `calc` is a pandas DataFrame, we can also easily calculate our own metrics and formulae using normal pandas syntax.

For example, for a sum assured of 10000, we can calculate a net premium by indexing the PVs at time 0


In [15]:
net_premium = 10000 * calc["EA(x1)[n1]"].iloc[0] / calc["a_due(x1)[n2]"].iloc[0]
net_premium


716.1739633598288

Often a better way is to add in the sum assured and premium as columns themselves into the calc dataframe ...


In [16]:
calc["sum_assured"] = 10000
calc["net_premium"] = net_premium
calc.head()


,q(x1),x1,i,n1,n2,v^t,l(x1),d(x1),C(x1),M(x1),D(x1),EA(x1)[n1],N(x1),a_due(x1)[n2],sum_assured,net_premium
0,0.000438,30,0.0325,15.0,10.0,1.000000,34481.408000,15.091478,14.616443,8719.096980,34481.408000,0.620982,818448.803936,8.670826,10000,716.173963
1,0.000574,31,0.0325,14.0,9.0,0.968523,34466.316522,19.773670,18.548431,8704.480537,33381.420360,0.641007,783967.395936,7.923596,10000,716.173963
2,0.000699,32,0.0325,13.0,8.0,0.938037,34446.542852,24.071933,21.869597,8685.932106,32312.125042,0.661645,750585.975575,7.152716,10000,716.173963
3,0.000738,33,0.0325,12.0,7.0,0.908510,34422.470919,25.408258,22.357058,8664.062509,31273.166763,0.682927,718273.850533,6.357122,10000,716.173963
4,0.000790,34,0.0325,11.0,6.0,0.879913,34397.062660,27.175055,23.159018,8641.705451,30266.424310,0.704905,687000.683770,5.535314,10000,716.173963


We can now easily calculate a technical reserve using standard pandas syntax. This vectorised calculation in pandas is very fast ...


In [17]:
calc["V"] = (
    calc["sum_assured"] * calc["EA(x1)[n1]"]
    - calc["net_premium"] * calc["a_due(x1)[n2]"]
)
calc["V"].head(18)


0         0.000000
1       735.394777
2      1493.864670
3      2276.467530
4      3084.798024
5      3919.699896
6      4782.074416
7      5672.888549
8      6593.184090
9      7544.088555
10     8526.827611
11     8802.221510
12     9086.810726
13     9380.998404
14     9685.230024
15    10000.000000
16        0.000000
17        0.000000
Name: V, dtype: float64

## Further formulae

The purpose of the _actymath_ module is to make it fast to create the core actuarial formulae, and to do so in pandas which enables fast, scaleable calculations.

The current list of formulae available that you can call with `Calc.populate()` are shown in the `Calc.formulae` property


In [18]:
calc.formulae


{'a_due(x{life})[n{term_id}]': ' PV of annuity due (paid in advance) for term n. ',
 'a(x{life})[n{term_id}]': ' PV of annuity (paid in arrears) for term n. ',
 'A(x{life})[n{term_id}]': ' PV of a term assurance (paid in arrears) for term n. ',
 'E(x{life})[n{term_id}]': ' PV of a pure endowment for term n. ',
 'EA(x{life})[n{term_id}]': ' PV of an endowment assurance (paid in arrears) for term n. ',
 'NP(x{life})[n{term_id}]': ' Net Premium a term assurance for term n. ',
 'Ia_due(x{life})[n{term_id}]': ' PV of arithmetically increasing annuity due (paid in advance) for term n. ',
 'Ia(x{life})[n{term_id}]': ' PV of arithmetically increasing annuity (paid in arrears) for term n. ',
 'IA(x{life})[n{term_id}]': ' PV of arithmetically increasing term assurance (paid in arrears) for term n. ',
 'IE(x{life})[n{term_id}]': ' PV of arithmetically increasing pure endowment (paid in arrears) for term n. ',
 'IEA(x{life})[n{term_id}]': ' PV of arithmetically increasing endowment assurance (paid

## Performance

A timed example that:

- Extracts mortality data for two lives
- Creates a new `Calc` grid
- Initialises two different terms
- Populates many actuarial formulae on all those combinations of lives and terms

This creates a dataframe with around 40 columns in a few milliseconds


In [19]:
%%time

calc = Calc()

calc.add_life(age=20, qx=table.qx(age=20))
calc.add_life(age=25, qx=table.qx(age=25))

calc.add_term(n=60)
calc.add_term(n=45)

calc.add_i(rate=0.02)

for life, term in zip(['x1','x2'],['n1','n2']):
    calc.populate(f'A({life})[{term}]')
    calc.populate(f'E({life})[{term}]')
    calc.populate(f'EA({life})[{term}]')
    calc.populate(f'IA({life})[{term}]')
    calc.populate(f'a_due({life})[{term}]')
    calc.populate(f'a({life})[{term}]')
    calc.populate(f'Ia_due({life})[{term}]')


CPU times: user 13.9 ms, sys: 1.76 ms, total: 15.6 ms
Wall time: 14.5 ms


In [20]:
calc.shape


(102, 38)

In [21]:
calc.head()


,q(x1),x1,q(x2),x2,n1,n2,i,v^t,l(x1),d(x1),C(x1),M(x1),D(x1),A(x1)[n1],E(x1)[n1],EA(x1)[n1],R(x1),IA(x1)[n1],N(x1),a_due(x1)[n1],a(x1)[n1],S(x1),Ia_due(x1)[n1],l(x2),d(x2),C(x2),M(x2),D(x2),A(x2)[n2],E(x2)[n2],EA(x2)[n2],R(x2),IA(x2)[n2],N(x2),a_due(x2)[n2],a(x2)[n2],S(x2),Ia_due(x2)[n2]
0,0.000662,20,0.000471,25,60.0,45.0,0.02,1.000000,34481.408000,22.836347,22.388575,11982.368463,34481.408000,0.249265,0.112004,0.361269,617618.795300,17.280201,1.147451e+06,32.575269,31.687273,2.702144e+07,780.048439,34481.408000,16.250743,15.932101,13139.442249,34481.408000,0.152281,0.285489,0.437769,618474.138418,15.476695,1.088440e+06,28.673760,27.959249,2.396827e+07,673.050313
1,0.000737,21,0.000572,26,59.0,44.0,0.02,0.980392,34458.571653,25.404237,24.417760,11959.979887,33782.913385,0.253756,0.114320,0.368076,605636.426837,17.282799,1.112970e+06,32.228118,31.342438,2.587399e+07,762.211274,34465.157257,19.711657,18.946230,13123.510149,33789.369860,0.154928,0.291336,0.446264,605334.696169,15.404810,1.053959e+06,28.240545,27.531881,2.287983e+07,654.622512
2,0.000797,22,0.000656,27,58.0,43.0,0.02,0.961169,34433.167416,27.456663,25.873027,11935.562128,33096.085559,0.258285,0.116692,0.374977,593676.446950,17.280089,1.079187e+06,31.876181,30.992873,2.476102e+07,744.400658,34445.445600,22.590012,21.287073,13104.563919,33107.886966,0.157545,0.297332,0.454877,592211.186020,15.325511,1.020169e+06,27.801256,27.098589,2.182587e+07,636.262995
3,0.000758,23,0.000647,28,57.0,42.0,0.02,0.942322,34405.710752,26.079873,24.093771,11909.689101,32421.269678,0.262863,0.119121,0.381984,581740.884822,17.271617,1.046091e+06,31.518837,30.637958,2.368184e+07,726.608223,34422.855588,22.267801,20.572006,13083.276846,32437.425639,0.160145,0.303478,0.463623,579106.622101,15.238285,9.870616e+05,27.355222,26.658700,2.080570e+07,617.963771
4,0.000724,24,0.000646,29,56.0,41.0,0.02,0.923845,34379.630880,24.879164,22.533825,11885.595329,31761.464737,0.267565,0.121596,0.389160,569831.195722,17.255441,1.013669e+06,31.152828,30.274424,2.263575e+07,708.766763,34400.587787,22.220028,20.125364,13062.704840,31780.825679,0.162806,0.309748,0.472554,566023.345255,15.141440,9.546242e+05,26.899727,26.209475,1.981864e+07,599.672660
